In [61]:
import cloudpickle as pickle
import os
from tqdm.auto import tqdm
import numpy as np
import torch
os.environ['CUDA_VISIBLE_DEVICES'] = "-1"

In [4]:
ckpt = '/home/sergei/ray_results/ve5_linear_tune_dimension_single_sample/main_fcn_d5c73_00000_0__gin__fdim__choice=6_2020-12-06_16-08-02/checkpoint_0/checkpoint'

In [5]:
learner = pickle.load(open(ckpt, 'rb'))

Make environment VectorIncrement-v0 None {}
Using device cpu


In [37]:
for _ in tqdm(range(100)):
    _ = learner._epoch()

In [38]:
learner._epoch()

{'epochs': 523,
 'n_samples': 1020,
 'losses/opt1/fit/coeff': 1.0,
 'losses/opt1/fit/value': 0.0037429044023156166,
 'losses/opt1/sparsity/coeff': 1e-09,
 'losses/opt1/sparsity/value': 12.967931747436523,
 'losses/opt1/reconstruction/coeff': 1.0,
 'losses/opt1/reconstruction/value': 1.0043048858642578,
 'losses/opt1/value': 1.0080478191375732,
 'weights/model/fc_features.weight': array([[ 0.5801595 ,  0.27174062,  0.5523837 , -0.29568988, -0.10499316,
          0.2771616 ],
        [-0.2252443 ,  0.3229994 , -0.23914137,  0.28891942,  0.1835801 ,
         -0.29170838],
        [ 0.19559123, -0.22587743,  0.6236473 ,  0.13790984,  0.09509419,
         -0.07009463],
        [-0.10744345,  0.25923783,  0.25487638,  0.79165804, -0.07313193,
          0.20357406],
        [-0.15406309,  0.41632557,  0.40683052, -0.27232787,  0.8579692 ,
          0.25745967],
        [ 0.15526381, -0.24550784, -0.2951532 ,  0.20269157,  0.07482588,
          0.7944947 ]], dtype=float32),
 'weights/model/fc_

In [48]:
obs = learner._context['obs']
obs

tensor([[0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.2000],
        [0.0000, 0.0000, 0.0000, 0.2000, 0.2000],
        ...,
        [2.0000, 1.2000, 3.4000, 1.8000, 1.2000],
        [2.0000, 1.2000, 3.4000, 1.8000, 1.4000],
        [2.0000, 1.2000, 3.4000, 2.0000, 1.4000]])

In [49]:
f = learner.decoder(obs)
f

tensor([[-1.6582,  1.5992, -1.6326, -1.2058,  1.2749, -1.5589],
        [-1.6578,  1.6182, -1.5643, -0.8914,  1.2316, -1.3859],
        [-1.6127,  1.5013, -1.5190, -0.8859,  1.2603, -1.2226],
        ...,
        [ 2.1617, -2.2604,  0.9817, -0.7656, -2.8950,  1.0151],
        [ 2.1621, -2.2414,  1.0500, -0.4512, -2.9383,  1.1881],
        [ 2.2071, -2.3583,  1.0953, -0.4457, -2.9096,  1.3515]],
       grad_fn=<NativeBatchNormBackward>)

In [60]:
np.mean(np.abs((learner.reconstructor(f) - obs).cpu().detach().numpy()))

1.0000001

In [79]:
learner.decoder

LinearDecoder(
  (fc): Linear(in_features=5, out_features=6, bias=False)
  (bn): BatchNorm1d(6, eps=1e-05, momentum=0.1, affine=False, track_running_stats=True)
)

In [86]:
adam = torch.optim.Adam(params=list(learner.decoder.parameters()) + list(learner.reconstructor.parameters()))

In [87]:
m = torch.nn.Linear(in_features=5, out_features=5)
adam = torch.optim.Adam(params=m.parameters())

In [88]:
for _ in range(100000):
    loss = torch.mean(torch.square(learner.reconstructor(learner.decoder(obs)) - obs))
#     loss = torch.mean(torch.square(obs - m(obs)))
    adam.zero_grad()
    loss.backward()
    adam.step()
    print(loss)

tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)
tensor(1.0034, grad_fn=<MeanBackward0>)


KeyboardInterrupt: 